# Neural networks

{{ badge }}


In [3]:
# !pip install torch onnxscript onnx netron torchinfo

## Create your own neural network

In [4]:
from torch import nn
import torch


# My NN
class MyNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(700, 200)
        self.a1 = nn.ReLU()
        self.l2 = nn.Linear(200, 100)
        self.a2 = nn.SELU()
        self.d2 = nn.Dropout(p=0.3)
        self.l3 = nn.Linear(100, 1)

    def forward(self, x):
        x = self.l1(x)
        x = self.a1(x)
        x = self.l2(x)
        x = self.a2(x)
        x = self.d2(x)
        x = self.l3(x)
        return x


model = MyNeuralNetwork()
print(model)

MyNeuralNetwork(
  (l1): Linear(in_features=700, out_features=200, bias=True)
  (a1): ReLU()
  (l2): Linear(in_features=200, out_features=100, bias=True)
  (a2): SELU()
  (d2): Dropout(p=0.3, inplace=False)
  (l3): Linear(in_features=100, out_features=1, bias=True)
)


## Visualize neural networks

See input and output shapes

In [5]:
from torchinfo import summary

batch_size = 250
inp = torch.randn(
    (
        batch_size,
        700,
    ),
    dtype=torch.float,
)
out = model(inp)


summary(
    model,
    input_size=inp.shape,
    device="cpu",
    col_names=("input_size", "output_size", "num_params"),
)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
MyNeuralNetwork                          [250, 700]                [250, 1]                  --
├─Linear: 1-1                            [250, 700]                [250, 200]                140,200
├─ReLU: 1-2                              [250, 200]                [250, 200]                --
├─Linear: 1-3                            [250, 200]                [250, 100]                20,100
├─SELU: 1-4                              [250, 100]                [250, 100]                --
├─Dropout: 1-5                           [250, 100]                [250, 100]                --
├─Linear: 1-6                            [250, 100]                [250, 1]                  101
Total params: 160,401
Trainable params: 160,401
Non-trainable params: 0
Total mult-adds (M): 40.10
Input size (MB): 0.70
Forward/backward pass size (MB): 0.60
Params size (MB): 0.64
Estimated Total Size (MB): 1.94

This might not run in Collab or locally without adjusting the URL

In [6]:
import IPython
import netron

# Onnx is a format for saving neural networks supported by pytroch, tensorflow, and other frameworks
torch.onnx.export(
    model, inp, "model.onnx", input_names=["Input"], output_names=["Output"]
)


port = 8053
model_path = "model.onnx"
netron.start(model_path, ("0.0.0.0", port), browse=False, verbosity=2)

IPython.display.IFrame(
    f"http://galadriel.ing.uni-bayreuth.de:8053/", width=800, height=800
)

Serving 'model.onnx' at http://0.0.0.0:8053


## Train

In [7]:
import pandas as pd

df = pd.read_json(
    "https://raw.githubusercontent.com/kuennethgroup/colab_tutorials/main/lecture2/data/polymers_tend_to_crystalize.json"
)

from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.20, random_state=42)
len(df), len(df_train), len(df_test)

(432, 345, 87)

In [16]:
train_tensor = torch.data_utils.TensorDataset(train, train_target)

,smiles,property,value,fingerprint
132,[*]OC(C)C([*])=O,Xc,57.620000,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
231,[*]CCCOC(=O)NCCCCCCNC(=O)O[*],Xc,28.370000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, ..."
31,[*]CC([*])C(=O)NCCCCCCCCCCCCCCCCCC,Xc,21.610000,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
84,[*]CCOC(=O)CCC(=O)O[*],Xc,79.990000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, ..."
296,[*]CC(CS(C)(=O)=O)O[*],Xc,0.370000,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
71,[*]Oc1ccc(Oc2ccc(OC(=O)c3cccc(C([*])=O)c3)cc2)cc1,Xc,29.770000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
106,[*]CCCCCOC(=O)c1ccc(C(=O)O[*])cc1,Xc,21.110000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, ..."
270,[*]C=CCCCCCCCCCC(CCCCCCCCC[*])COCCOCCOCCOCCOCC...,Xc,8.900000,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, ..."
348,[*]C=Cc1cccc(OC(=O)CCCCCCCCC(=O)Oc2cccc(-c3nnc...,Xc,12.490000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [15]:
trainloader = torch.utils.data.DataLoader(
    df_train, batch_size=batch_size, shuffle=True, num_workers=2
)
list(trainloader)

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/chris/courses/colab_tutorials/.venv/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 323

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/chris/courses/colab_tutorials/.venv/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/chris/courses/cola

In [10]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')